### QV to Ore

In [ ]:
import qiskit.ignis.verification.quantum_volume as qv
from qiskit2ore.dump import dump_qiskit_circuit_assets

# qv_circuits call requires connectivitiy to build circuits, will return QV circuits
# between min_qubits and max_qubits

# qiskit's qv_circuits algorithm will generate a batch of circuit for
# each width from min_qubits to max_qubits.
min_qubits=2
max_qubits=6
# number of circuits for each qubit width. QV circuits are random,
# and generally need many variants to approximate uniform unitaries.
max_circuits=1

print("converting qv to ore")

qubit_lists = [list(range(i)) for i in range(min_qubits, max_qubits + 1)]
_qv_circs, qv_circs_nomeas = qv.qv_circuits(qubit_lists, max_circuits)
for circuit_group in qv_circs_nomeas:
    for circuit in circuit_group:
        dump_qiskit_circuit_assets(circuit, preprocessor=True)

print("done")

## AE to Ore

In [ ]:
import sys
sys.path.insert(1, "amplitude-estimation/qiskit")
from ae_benchmark import AmplitudeEstimation
from qiskit2ore.dump import dump_qiskit_circuit_assets
import numpy as np


##### PARAMETERS
min_qubits=3
max_qubits=8
max_circuits=3
preprocessor=True

# default, not exposed to users
num_state_qubits=1

def a_from_s_int(s_int, num_counting_qubits):
    theta = s_int * np.pi / (2**num_counting_qubits)
    precision = int(num_counting_qubits / (np.log2(10))) + 2
    a = round(np.sin(theta)**2, precision)
    return a

MAX_QUBITS = 8

# Clamp the maximum number of qubits
if max_qubits > MAX_QUBITS:
    print(f"INFO: Amplitude Estimation benchmark is limited to a maximum of {MAX_QUBITS} qubits.")
    max_qubits = MAX_QUBITS

# validate parameters (smallest circuit is 3 qubits)
num_state_qubits = max(1, num_state_qubits)
if max_qubits < num_state_qubits + 2:
    raise TypeError(f"ERROR: AE Benchmark needs at least {num_state_qubits + 2} qubits to run")
min_qubits = max(max(3, min_qubits), num_state_qubits + 2)


# Execute Benchmark Program N times for multiple circuit sizes
# Accumulate metrics asynchronously as circuits complete
for num_qubits in range(min_qubits, max_qubits + 1):
    
    # as circuit width grows, the number of counting qubits is increased
    num_counting_qubits = num_qubits - num_state_qubits - 1

    # determine number of circuits to execute for this group
    num_circuits = min(2 ** (num_counting_qubits), max_circuits)
    
    print(f"************\nExecuting [{num_circuits}] circuits with num_qubits = {num_qubits}")
    
    # determine range of secret strings to loop over
    if 2**(num_counting_qubits) <= max_circuits:
        s_range = list(range(num_circuits))
    else:
        s_range = np.random.choice(2**(num_counting_qubits), num_circuits, False)
    
    # loop over limited # of secret strings for this
    for s_int in s_range:
        # create the circuit for given qubit size and secret string
        a_ = a_from_s_int(s_int, num_counting_qubits)

        qc = AmplitudeEstimation(num_state_qubits, num_counting_qubits, a_)
        dump_qiskit_circuit_assets(qc, preprocessor)
        

## VQE to Ore

In [1]:
import sys
sys.path.insert(1, "vqe/qiskit")
from vqe_circuits import generate_circuits
from qiskit2ore.dump import dump_qiskit_circuit_assets

##### PARAMS
min_qubits=4
max_qubits=10
max_circuits=3
method=1
preprocessor=True

circuits = generate_circuits(min_qubits, max_qubits, max_circuits, method)
for circuit in circuits:
    dump_qiskit_circuit_assets(circuit, preprocessor)

Variational Quantum Eigensolver Benchmark
... using circuit method 1
************
Executing [3] circuits with num_qubits = 4
